<a href="https://colab.research.google.com/github/lrhf97/OpenCV/blob/main/image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, MaxPool2D, Dropout, Conv2D, Flatten
from keras.constraints import maxnorm
from keras.datasets import cifar10

# import matplotlib.pyplot as plt
import numpy as np


def reshape_image(input_image_arrays):
    output_array = []
    for image_array in input_image_arrays:
        output_array.append(image_array.reshape(-1))
    return np.asarray(output_array)


labels_array = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(len(x_train))

# first_image = x_train[0]
# print image
# print(first_image[0])
# plt.imshow(first_image)
# plt.show()
# max_label_index = np.argmax(train_labels[0])
# print(labels_array[max_label_index])

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# print(x_test[0][0])


Epoch 1/10
1563/1563 [==============================] - 107s 68ms/step - loss: 1.9324 - accuracy: 0.3085
Epoch 2/10
1563/1563 [==============================] - 103s 66ms/step - loss: 1.6603 - accuracy: 0.4129
Epoch 3/10
1563/1563 [==============================] - 103s 66ms/step - loss: 1.5029 - accuracy: 0.4664
Epoch 4/10
1563/1563 [==============================] - 103s 66ms/step - loss: 1.3932 - accuracy: 0.5054
Epoch 5/10
1563/1563 [==============================] - 103s 66ms/step - loss: 1.3113 - accuracy: 0.5336
Epoch 6/10
1563/1563 [==============================] - 99s 63ms/step - loss: 1.2502 - accuracy: 0.5551
Epoch 7/10
1563/1563 [==============================] - 99s 63ms/step - loss: 1.1995 - accuracy: 0.5756
Epoch 8/10
1563/1563 [==============================] - 99s 63ms/step - loss: 1.1582 - accuracy: 0.5929
Epoch 9/10
1563/1563 [==============================] - 99s 63ms/step - loss: 1.1166 - accuracy: 0.6049
Epoch 10/10
1563/1563 [==============================] - 99

In [ ]:

model = Sequential()

# filters; output 32 features (the image is 32hx32wx3colors
# kernel_size : 3x3 kernel or filter matrix used to calculate output features
# input_shape: each input is 32x32x3
# activation: relu is the best for this
# padding: 'same' adds padding to the input image to make sure that the output feature map is the same size as the input
# kernel_restraint: maxnorm normalizes the values in the kernel to make that the max value is 3
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same', kernel_constraint=maxnorm(3)))

# Add the max pool layer to decrease the image size form 2x32 to 16x16
# pool_size: finds the max value oin each 2x2 section of the input
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())

# First dense layer to create the actual prediction network
# units: 512 neurons at this layer, increase for greater accuracy, decrease for faster train speed
# kernel_constraint: see above
model.add(Dense(512, activation='relu', kernel_constraint=maxnorm(3)))

# Dropout layer to ignore some neurons during training which improves model reliability
# rate: 0.5 means half the neurons dropped
model.add(Dropout(rate=0.5))

# Final dense layer used to produce output for each of the 10 categories
# units: 10 categories so 10 output units
# activation: softmax because we are calculating probabilities for each of the 10 categories (not as clear as 0 or 1)
model.add(Dense(10, activation='softmax'))

opt = tf.keras.optimizers.SGD(learning_rate=0.01)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x=x_train, y=y_train, epochs=10, batch_size=32)


Epoch 1/10
1563/1563 [==============================] - 12s 7ms/step - loss: 1.9135 - accuracy: 0.3146
Epoch 2/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.6238 - accuracy: 0.4246
Epoch 3/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.4782 - accuracy: 0.4735
Epoch 4/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.3786 - accuracy: 0.5080
Epoch 5/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.2972 - accuracy: 0.5400
Epoch 6/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.2404 - accuracy: 0.5594
Epoch 7/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.1931 - accuracy: 0.5775
Epoch 8/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.1456 - accuracy: 0.5959
Epoch 9/10
1563/1563 [==============================] - 11s 7ms/step - loss: 1.1065 - accuracy: 0.6100
Epoch 10/10
1563/1563 [==============================] - 11s 7ms/step - l